In [1]:
# Should be first to load all the env vars for browser
import os
from dotenv import load_dotenv

load_dotenv()

assert os.getenv("CHROME_USER_DATA_DIR", None)

# Needed to use the static/ default folder with the cache
from browser import BRAVE_SEARCH_SESSION

BRAVE_SEARCH_SESSION.browser.chrome_user_data_dir

sh: pidof: command not found


'/Users/maksim.rostov/pdev/freelectron/free-generalist/static/browser_cache'

In [2]:
import mlflow
import logging

mlflow.langchain.autolog()   # this is needed to register traces within the experiment
experiment_name = f"agent_test"
mlflow.set_experiment(experiment_name)
logging.getLogger().setLevel(logging.INFO)

In [3]:
from generalist.agents.core import AgentUnstructuredDataProcessor, AgentDeepWebSearch
from generalist.tools.data_model import Message

content_resources = Message(
        provided_by= "deep_web_search",
        content= "THE FILE",
        link= "/Users/maksim.rostov/pdev/freelectron/free-generalist/src/browser/browser.py",
        metadata= {}
)

task = "Analyse the file"

## Test Processing
agent = AgentUnstructuredDataProcessor(activity=task)
out = agent.run([content_resources])

# Web Search
# agent = AgentDeepWebSearch(activity=task)
# out = agent.run()

/Users/maksim.rostov/pdev/freelectron/free-generalist/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


chunks ['import os\nfrom time import sleep\nfrom typing import Any, Dict, Optional\n\nimport undetected as uc\nfrom selenium.webdriver.support.ui import WebDriverWait\n\nfrom clog import get_logger\n\n\nclass ChromeBrowser:\n waiter_default_timeout = 10\n # Mapping from tabs tittle to their window handles\n opened_tabs: Dict[str, Any] = {}\n\n @staticmethod\n def wait(seconds: float = 1):\n sleep(seconds)\n\n @staticmethod\n def get_default_options():\n """Return default Chrome options."""\n options = uc.ChromeOptions()\n options.add_argument("--no-sandbox")\n options.add_argument("--disable-blink-features=AutomationControlled")\n\n # options.add_argument("--user-agent=Chrome/122.0.0.0")\n options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")\n\n return options\n\n def __init__(self, profile: Optional[str] = None):\n self.logger = get_logger(name=self.__class__.__name__)\n\n self.chrome_user_

In [4]:
print(out)

Message(provided_by='unstructured_data_processing', content='Query: Analyse the file \n Answers: ["The context provided is a partial implementation of a `ChromeBrowser` class in Python using Selenium and Undetected Chrome (uc) to automate web browser interactions. Here\'s an analysis based on the given information:\\n\\n1. **Imports**: The file imports necessary modules from libraries such as `os`, `time`, `typing`, `selenium.webdriver.support.ui`, and a custom logger module (`clog`). Additionally, it uses Undetected Chrome for running Selenium without detection.\\n\\n2. **Class Attributes**:\\n   - `waiter_default_timeout`: A static attribute set to 10 seconds.\\n   - `opened_tabs`: An empty dictionary used as a mapping from tab titles to their window handles.\\n\\n3. **Static Methods**:\\n   - `wait(seconds: float = 1)`: Sleeps for the given number of seconds, defaulting to 1 second if not specified.\\n   \\n4. **Class Method**:\\n   - `get_default_options()`: Returns a set of Chrome